In [34]:
import sys
import os
import warnings
import traceback
import logging
import time
import dotenv
from pyspark import SparkConf, SparkContext
dotenv.load_dotenv()

False

In [7]:
# from utils.helpers import load_cfg 
import yaml


def load_cfg(cfg_file):
    """
    Load configuration from a YAML config file
    """
    cfg = None
    with open(cfg_file, "r") as f:
        try:
            cfg = yaml.safe_load(f)
        except yaml.YAMLError as exc:
            print(exc)

    return cfg


In [8]:
# from utils.minio_utils import MinIOClient

from minio import Minio


class MinIOClient:
    def __init__(self, endpoint_url, access_key, secret_key):
        self.endpoint_url = endpoint_url
        self.access_key = access_key
        self.secret_key = secret_key

    def create_conn(self):
        client = Minio(
            endpoint=self.endpoint_url,
            access_key=self.access_key,
            secret_key=self.secret_key,
            secure=False,
        )
        return client

    def create_bucket(self, bucket_name):
        client = self.create_conn()

        # Create bucket if not exist
        found = client.bucket_exists(bucket_name=bucket_name)
        if not found:
            client.make_bucket(bucket_name=bucket_name)
            print(f"Bucket {bucket_name} created successfully!")
        else:
            print(f"Bucket {bucket_name} already exists, skip creating!")

    def list_parquet_files(self, bucket_name, prefix=""):
        client = self.create_conn()

        # List all objects in the bucket with the given prefix
        objects = client.list_objects(bucket_name, prefix=prefix, recursive=True)
        # Filter and collect Parquet file names
        parquet_files = [
            obj.object_name for obj in objects if obj.object_name.endswith(".parquet")
        ]

        return parquet_files


In [9]:
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s:%(funcName)s:%(levelname)s:%(message)s')
warnings.filterwarnings('ignore')

In [10]:
POSTGRES_USER = os.getenv("DB_USER")
POSTGRES_PASSWORD = os.getenv("DB_PASSWORD")
POSTGRES_HOST = os.getenv("DB_HOST")
POSTGRES_PORT = os.getenv("DB_PORT")
DB_STAGING_TABLE = os.getenv("DB_STAGING_TABLE")

In [11]:
__file__ = os.getcwd()
print(__file__) 

d:\BigData\MyProject\src\batch_processing


In [12]:
project_root = os.path.dirname(
    os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
)
project_root

'd:\\BigData'

In [23]:
CFG_FILE = os.path.join(project_root, "MyProject/config", "datalake.yaml")

In [28]:
cfg = load_cfg(CFG_FILE)
datalake_cfg = cfg["datalake"]
datalake_cfg

{'endpoint': 'localhost:9000',
 'bucket_name_1': 'raw',
 'bucket_name_2': 'processed',
 'bucket_name_3': 'sandbox',
 'folder_name': 'batch',
 'access_key': '0VQBMtMhycuIrat2ivLH',
 'secret_key': 'xozBRG1AkxBkEnwN3JePy1BhhvHQGtE1sCAEmZeI'}

In [29]:
CFG_FILE_SPARK = os.path.join(project_root, "MyProject/config", "spark.yaml")
spark_cfg = load_cfg(CFG_FILE_SPARK)[ "spark_config"]
spark_cfg

{'executor_memory': '4g'}

In [31]:
MEMORY = spark_cfg["executor_memory"]

In [32]:
MINIO_ENDPOINT = datalake_cfg["endpoint"]
MINIO_ACCESS_KEY = datalake_cfg["access_key"]
MINIO_SECRET_KEY = datalake_cfg["secret_key"]
BUCKET_NAME_2 = datalake_cfg['bucket_name_2']
BUCKET_NAME_3 = datalake_cfg['bucket_name_3']

In [33]:
def create_spark_session():
    """
    Create a Spark session
    """
    from pyspark.sql import SparkSession

    try: 
        spark = (SparkSession.builder.config("spark.executor.memory", MEMORY) \
                        .config(
                            "spark.jars", 
                            "jars/postgresql-42.4.3.jar,jars/aws-java-sdk-bundle-1.12.262.jar,jars/hadoop-aws-3.3.4.jar",
                        )
                        .config("spark.sql.execution.arrow.pyspark.enabled", "true")
                        .appName("Batch Processing Application")
                        .getOrCreate()
        )
        
        logging.info('Spark session successfully created!')

    except Exception as e:
        traceback.print_exc(file=sys.stderr)
        logging.error(f"Couldn't create the spark session due to exception: {e}")

    return spark

In [35]:
def load_minio_config(spark_context: SparkContext):
    """
        Establist the necessary connection to MinIO
    """
    try:
        spark_context._jsc.hadoopConfiguration().set("fs.s3a.access.key", MINIO_ACCESS_KEY)
        spark_context._jsc.hadoopConfiguration().set("fs.s3a.secret.key", MINIO_SECRET_KEY)
        spark_context._jsc.hadoopConfiguration().set("fs.s3a.endpoint", MINIO_ENDPOINT)
        spark_context._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
        spark_context._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
        spark_context._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
        spark_context._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        logging.info('MinIO configuration is created successfully')
    except Exception as e:
        traceback.print_exc(file=sys.stderr)
        logging.error(f"Couldn't create the MinIO configuration due to exception: {e}")

In [ ]:
def processing_dataframe(df, file_path):
    """
    Processing the dataframe
    """
    from pyspark.sql import functions as F 
    
    df2 = df.withColumn('year', F.year("pickup_datetime")) \
            .withColumn("month", F.date_format("pickup_datetime", "MMM")) \
            .withColumn("dow", F.date_format("pickup_datetime", "EEEE"))
            
    df_final = df2.groupBy(
        "year",
        "month",
        "dow",
        F.col("vendor_id").alias("vendor_id"),
        F.``
    )
                

In [1]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("Spark to BigQuery") \
    .getOrCreate()

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.UnsupportedOperationException: getSubject is supported only if a security manager is allowed
	at java.base/javax.security.auth.Subject.getSubject(Subject.java:347)
	at org.apache.hadoop.security.UserGroupInformation.getCurrentUser(UserGroupInformation.java:577)
	at org.apache.spark.util.Utils$.$anonfun$getCurrentUserName$1(Utils.scala:2416)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.util.Utils$.getCurrentUserName(Utils.scala:2416)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:329)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:501)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.ja

In [1]:
import pandas as pd

In [2]:
taxi_lookup = pd.read_csv("../../data/taxi_lookup.csv")
taxi_lookup

,LocationID,Borough,zone,service_zone,latitude,longitude
0,1,EWR,Newark Airport,EWR,40.689064,-74.177255
1,2,Queens,Jamaica Bay,Boro Zone,40.603994,-73.835412
2,3,Bronx,Allerton,Boro Zone,39.915319,-87.933215
3,4,Manhattan,Alphabet City,Yellow Zone,40.725102,-73.979583
4,5,Staten Island,Arden Heights,Boro Zone,53.284320,-7.492801
...,...,...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone,40.711900,-74.012527
261,262,Manhattan,Yorkville East,Yellow Zone,33.465410,-88.358724
262,263,Manhattan,Yorkville West,Yellow Zone,36.099513,-89.118957
263,264,Unknown,NV,NaN,39.515882,-116.853722


In [3]:
taxi_lookup.to_parquet("taxi_lookup.parquet")